# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [172]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from word2number import w2n
from pandas import json_normalize

In [108]:
#Variables
url_list = []
page_number = 1
min_rating = 1.0
max_price = 5

In [106]:
#URLS
base_url = 'https://books.toscrape.com/catalogue/'
current_url = ''
catalogue_url = 'https://books.toscrape.com/catalogue/page-'+str(page_number)+'.html'

In [32]:
def get_new_page(page_url):
    """
    ****DO NOT CALL****
    This function takes a url and requests it. It is used inside get_soup()
    """
    response = requests.get(page_url)
    return response

In [87]:
def next_page():
    """
    This function updates the page iterator for the catalogue
    """
    global page_number
    page_number += 1

In [83]:
def get_url_grid(url):
    """
    This function is used to populate the list of links to each book where we are going to extract the information
    """    
    soup = BeautifulSoup(get_new_page(url).content)
    next_page()
    grid = soup.find_all('article', attrs = {'class':'product_pod'})
    return grid

In [ ]:
def get_product_details(url):
    """
    This function is used to extract the information from the product page
    """
    try:
        book = {}
        response = requests.get(url)
        soup = BeautifulSoup(response.content)
        page = soup.find_all('div', attrs = {'class':'page_inner'})
        title = page[1].find('h1').get_text().strip()
        category = page[1].find_all('a')[2].get_text().strip()
        description = page[1].find_all(['p'])[3].get_text().strip()
        price = float(page[1].find_all('p', attrs = {'class':'price_color'})[0].get_text().strip().replace('£',''))
        rating = w2n.word_to_num(page[1].find_all('p')[2]['class'][1])
        id = page[1].find_all('td')[0].get_text().strip()
        availability = page[1].find_all('td')[5].get_text().strip()
        href = url
        
        book[id] = {'title': title,
                    'category': category,
                    'description': description,
                    'star_rating': rating,
                    'price': price,
                    'availability':availability,
                    'href': url}
        return book     
    except:
        print(url)

In [18]:
def get_href(book):
    href = book.find_all('a')[0]['href']
    return href

In [23]:
def get_urls():
    url_grid = get_url_grid(catalogue_url)
    url_list = []
    for book in url_grid:
        new_url = base_url+get_href(book)
        url_list.append(new_url)
    return url_list

In [91]:
def get_books():
    url_list = get_urls()
    book_dict = {}
    index = 0
    for url in url_list:
        book_dict[index] = get_product_details(url)
        index += 1
    
    return book_dict